In [ ]:
import requests
import time
import pandas as pd

# GraphQL API endpoint
url = 'https://hub.snapshot.org/graphql'


# Function to handle rate limits and make requests
def make_request(query, variables=None):
    successful_request = False
    while not successful_request:
      try:
        response = requests.post(url, json={'query': query, 'variables': variables})
        if response.status_code == 200:
            return response.json()
      except Exception as e:
          print(f"Request failed: {e}. Retrying in 30 seconds...")
          time.sleep(60)  # Wait for 30 seconds before retrying

# Function to fetch paginated data
def fetch_paginated_data(query_template, key, extra_variables={}):
    all_data = []
    first = 1000
    created_gte = 0
    request_count = 0
    start_time = time.time()


    while True:
        if(key=="votes"):
          query = query_template.format(first=first, created_gte=created_gte, **extra_variables)
        else:
          query = query_template.format(first=first,**extra_variables)
        data = make_request(query)
        try:
            items = data['data'][key]
            if items is None:
                print(f"Warning: No items found for key '{key}'.")
                break

        except KeyError:
            print("KeyError: The structure of the response might not include 'data' as expected.")
            break

        all_data.extend(items)
        if(key=="votes"):
          # print(query)
          created_gte = items[-1]['created']
        if len(items) < first:
            break

    deduplicated_data = list({item['id']: item for item in all_data}.values())
    return deduplicated_data

# Template for fetching proposals
proposals_query_template = """
query Proposals {{
  proposals(
    first: {first},
    where: {{
      space_in: ["arbitrumfoundation.eth"]
    }},
    orderBy: "created",
    orderDirection: desc
  ) {{
    id
    title
    body
    choices
    start
    end
    snapshot
    state
    scores
    scores_by_strategy
    scores_total
    scores_updated
    author
  }}
}}
"""

# Template for fetching votes
votes_query_template = """
query Votes {{
  votes (
    first: {first},
    where: {{
      proposal: "{proposal_id}",
      created_gte: {created_gte}
    }},
    orderBy: "created",
    orderDirection: asc
  ) {{
    id
    voter
    created
    choice
    vp
    vp_by_strategy
    vp_state
    space {{
      id
    }}
  }}
}}
"""

# Fetch all proposals
proposals = fetch_paginated_data(proposals_query_template, 'proposals')

# DataFrame to store all votes
all_votes = []

# Fetch votes for each proposal
for proposal in proposals:
    proposal_id = proposal['id']
    print(f" Proposal: {proposal['id']}")
    votes = fetch_paginated_data(votes_query_template, 'votes', {'proposal_id': proposal_id})
    for vote in votes:
        vote['proposal_id'] = proposal_id  # Add proposal_id to each vote for reference
        # vp_data = fetch_voting_power(vote['voter'], proposal_id, "arbitrumfoundation.eth")  # Ensure you use the correct space ID
        # if vp_data:
        #     vote.update({
        #         'vp': vp_data['vp'],
        #         'vp_by_strategy': vp_data['vp_by_strategy'],
        #         'vp_state': vp_data['vp_state'],
        #     })
    all_votes.extend(votes)

# Convert to pandas DataFrames
df_proposals = pd.DataFrame(proposals)
df_votes = pd.DataFrame(all_votes)

# Print DataFrame shapes as a simple verification
print(f"Proposals DataFrame shape: {df_proposals.shape}")
print(f"Votes DataFrame shape: {df_votes.shape}")

df_proposal_summary = df_proposals[["id","title","choices","state","scores","scores_by_strategy","scores_total","scores_updated"]]
df_votes.to_csv('arb_foundation_proposals_space_votes.csv', index=False)